In [44]:
import pandas as pd

In [45]:
# Read random forest result
result = pd.read_csv('RandomForest_result.csv')
result['census_tract_id'] = result['census_tract_id'].astype(str)
result

,Unnamed: 0,PROPERTY_ACRES,age_of_the_property,state_code,county_code,starbucks_within_om,census_tract_id,target_within_tm,census_tract,zip code,...,Travel time to work: 35-39 min,Travel time to work: 40-44 min,Travel time to work: 45-59 min,Travel time to work: 60-89 min,Travel time to work: 90+ min,total_population,competitors,Predictions,Actual,Difference
0,0,0.62,20849.0,8.0,31.0,3.0,8031004005.0,13.0,4005.0,80210.0,...,0.038339,0.014909,0.078807,0.024494,0.010117,3751.000000,1.0,1.945763,1.740000,-0.205763
1,1,7.60,16101.0,12.0,1.0,1.0,12001001522.0,2.0,1522.0,32607.0,...,0.014252,0.019779,0.045957,0.020652,0.013089,9216.000000,2.0,1.297336,1.605217,0.307882
2,2,0.70,1645.0,48.0,201.0,10.0,48201980700.0,5.0,980700.0,77002.0,...,0.037221,0.043838,0.094293,0.075269,0.026468,2299.000000,6.0,2.385190,2.745887,0.360697
3,3,11.51,1945.0,48.0,309.0,1.0,48309004300.0,1.0,4300.0,76711.0,...,0.005106,0.004085,0.000000,0.014041,0.023487,8059.000000,2.0,1.490236,1.459349,-0.030887
4,4,19.31,1062.0,12.0,81.0,0.0,12081001916.0,5.0,1916.0,33908.0,...,0.021383,0.035487,0.056415,0.044586,0.016379,5392.000000,4.0,2.171356,1.916439,-0.254918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11940,11940,6.13,14275.0,13.0,59.0,1.0,13059030200.0,1.0,30200.0,30605.0,...,0.009872,0.043929,0.036032,0.029615,0.030602,4734.000000,0.0,1.137750,0.713717,-0.424032
11941,11941,3.22,2155.0,49.0,35.0,0.0,49035111802.0,6.0,111802.0,84106.0,...,0.041404,0.009123,0.014035,0.025263,0.016842,2777.000000,3.0,2.003999,2.101669,0.097669
11942,11942,13.00,8431.0,1.0,111.0,0.0,1111012916.0,3.0,12916.0,35216.0,...,0.016279,0.017442,0.049419,0.022674,0.010465,3661.166667,0.0,1.121560,0.880000,-0.241560
11943,11943,7.14,2274.0,37.0,183.0,0.0,37183054113.0,6.0,54113.0,27604.0,...,0.002817,0.016432,0.084977,0.014554,0.004225,4887.000000,2.0,1.664469,1.368169,-0.296300


In [46]:
# Create a new column to calculate the % differences between prediction actual rent per square feet price
result['Percentage diff'] = (result['Predictions']/result['Actual']-1)*100
# Label differences > 10 (considered undervalued) as 1, else 0
result['Label'] = [1 if x > 10 else 0 for x in result['Percentage diff']]
result.columns

Index(['Unnamed: 0', 'PROPERTY_ACRES', 'age_of_the_property', 'state_code',
       'county_code', 'starbucks_within_om', 'census_tract_id',
       'target_within_tm', 'census_tract', 'zip code',
       'Total pop < high school', 'Total pop high school graduate',
       'Total pop w/ some college or associates', 'Total pop w/ bachelors',
       'Total pop w/ grad degree',
       'Total women 15 TO 50y who birthed in last 12m',
       'Total with household income <$10k',
       'Total with household income $10-15k',
       'Total with household income $15-20k',
       'Total with household income $20-25k',
       'Total with household income $25-30k',
       'Total with household income $30-35k',
       'Total with household income $35-40k',
       'Total with household income $40-45k',
       'Total with household income $45-50k',
       'Total with household income $50-60k',
       'Total with household income $60-75k',
       'Total with household income $75-100k',
       'Total with 

In [47]:
# Group By Census Tract
result_grouped = result.groupby(["census_tract_id"]).agg(
     undervalued = ('Label','sum'),
     total = ("Label", 'count')
     ).reset_index()
result_grouped["census_tract_id"] = result_grouped["census_tract_id"].astype(str)
# Ratio difference for # of undervalued properties / total properties
result_grouped['index'] = result_grouped['undervalued']/ result_grouped['total']
# Filter for total properties > 4 (see code cell after for explanation) & Index > 0.5
result_grouped.loc[(result_grouped['total']>4)&(result_grouped['index']>0.5)].sort_values(by='index', ascending = False)

,census_tract_id,undervalued,total,index
1715,13121011310.0,5,6,0.833333
2432,32019002505.0,4,5,0.800000
3585,4013319104.0,4,5,0.800000
6208,48439102000.0,4,5,0.800000
4914,48113002100.0,4,6,0.666667
5635,48201410601.0,5,8,0.625000
1358,13051010806.0,3,5,0.600000
1507,13089021204.0,3,5,0.600000
1890,13245010203.0,3,5,0.600000
2176,22033003912.0,3,5,0.600000


In [48]:
# Since the max total properties within a census tract is 9, we decided to only include census tract with total properties > 4, selecting total properties between 1-3 might not be as reliable as the number of observations are smaller
result_grouped['total'].max()

9

In [49]:
result_grouped

,census_tract_id,undervalued,total,index
0,1003010600.0,1,1,1.0
1,1003010704.0,0,1,0.0
2,1003010708.0,3,3,1.0
3,1003010709.0,1,1,1.0
4,1003010710.0,0,1,0.0
...,...,...,...,...
7157,8123002019.0,1,2,0.5
7158,8123002105.0,0,1,0.0
7159,8123002205.0,1,1,1.0
7160,8123002206.0,0,1,0.0


In [50]:
# Changing the census tract id format to remove the .00
result_grouped['census_tract_id'] = result_grouped['census_tract_id'].astype(str).str[:-2]
result_grouped['census_tract_id']

0       1003010600
1       1003010704
2       1003010708
3       1003010709
4       1003010710
           ...    
7157    8123002019
7158    8123002105
7159    8123002205
7160    8123002206
7161    8123002209
Name: census_tract_id, Length: 7162, dtype: object

Putting back the Property Longitude/Latitude for Visualization

In [51]:
# Read the dataset prior to splitting into training/test dataset
df_combined_ungrouped_final = pd.read_csv("df_combined_ungrouped_final.csv")
df_combined_ungrouped_final['census_tract_id']

0        48041001110
1        48041001111
2        48041004311
3        48041000205
4        48041010358
            ...     
23884    47093000100
23885    22071013400
23886    22071001737
23887    22071000619
23888    22051028800
Name: census_tract_id, Length: 23889, dtype: int64

In [52]:
# Convert census tract id to str for merge purposes
df_combined_ungrouped_final["census_tract_id"] = df_combined_ungrouped_final["census_tract_id"].astype(str)
df_combined_ungrouped_final["census_tract_id"]

0        48041001110
1        48041001111
2        48041004311
3        48041000205
4        48041010358
            ...     
23884    47093000100
23885    22071013400
23886    22071001737
23887    22071000619
23888    22051028800
Name: census_tract_id, Length: 23889, dtype: object

In [53]:
# Only including the census tract id and latitude/longitude
df_combined_ungrouped_final_map = df_combined_ungrouped_final[["census_tract_id", "PROPERTY_LATITUDE", "PROPERTY_LONGITUDE"]]
df_combined_ungrouped_final_map

,census_tract_id,PROPERTY_LATITUDE,PROPERTY_LONGITUDE
0,48041001110,31.829729,-106.525397
1,48041001111,31.811390,-106.508570
2,48041004311,31.781474,-106.339508
3,48041000205,31.886572,-106.407021
4,48041010358,31.821640,-106.243430
...,...,...,...
23884,47093000100,35.967200,-83.917880
23885,22071013400,29.948683,-90.074981
23886,22071001737,30.047655,-89.980910
23887,22071000619,29.920149,-90.023146


In [54]:
# Only including the census tract id and latitude/longitude
df_combined_ungrouped_final_map = pd.merge(df_combined_ungrouped_final_map, result_grouped, how='inner', on = 'census_tract_id')
df_combined_ungrouped_final_map

,census_tract_id,PROPERTY_LATITUDE,PROPERTY_LONGITUDE,undervalued,total,index
0,48041001110,31.829729,-106.525397,1,4,0.25
1,48041001110,31.833054,-106.525681,1,4,0.25
2,48041001110,31.830805,-106.525640,1,4,0.25
3,48041001110,31.834864,-106.525187,1,4,0.25
4,48041001110,31.836006,-106.524869,1,4,0.25
...,...,...,...,...,...,...
19513,47093004300,36.044786,-83.883675,0,1,0.00
19514,47093004300,36.035349,-83.879948,0,1,0.00
19515,47001020202,36.017625,-84.240111,0,1,0.00
19516,47093005708,35.872778,-84.095764,0,1,0.00


In [55]:
df_combined_ungrouped_final_map.drop_duplicates(inplace=True)

In [56]:
df_combined_ungrouped_final_map

,census_tract_id,PROPERTY_LATITUDE,PROPERTY_LONGITUDE,undervalued,total,index
0,48041001110,31.829729,-106.525397,1,4,0.25
1,48041001110,31.833054,-106.525681,1,4,0.25
2,48041001110,31.830805,-106.525640,1,4,0.25
3,48041001110,31.834864,-106.525187,1,4,0.25
4,48041001110,31.836006,-106.524869,1,4,0.25
...,...,...,...,...,...,...
19513,47093004300,36.044786,-83.883675,0,1,0.00
19514,47093004300,36.035349,-83.879948,0,1,0.00
19515,47001020202,36.017625,-84.240111,0,1,0.00
19516,47093005708,35.872778,-84.095764,0,1,0.00


In [57]:
# Check to make sure all latitude/longitude are merged
df_combined_ungrouped_final_map[["PROPERTY_LATITUDE", "PROPERTY_LONGITUDE"]].isnull().sum()

PROPERTY_LATITUDE     0
PROPERTY_LONGITUDE    0
dtype: int64

In [58]:
# Convert to CSV to be inputted in Tableau
# df_combined_ungrouped_final_map.to_csv ('df_combined_ungrouped_final_map.csv') 

In [59]:
# Finding the median ret per square feet price for visualization threshold
df_combined_ungrouped_final["rent_per_sqft"].median()

1.449922058